In [1]:
from tfqml import *
import pennylane as qml

ModuleNotFoundError: No module named 'tensorflow_quantum'

In [ ]:
name_stocks = ['AAPL','ABB','ABBV','TOT','WMT','DUK','CHL','HSBC']

In [ ]:
index = 2
cp = ClassicalPreprocessing(filename=name_stocks[index])
days = 30
x_train,y_train,x_test,y_test = cp.preprocessing(days=days,flag_pca=True)

In [ ]:
n_qubits = days//3

dev = qml.device("default.qubit.tf", wires=n_qubits) # using backend tensorflow


#quantum circuit for the encoding and ansayz

@qml.qnode(dev)
def qnode(inputs,weights_0,weights_1,weights_2,weights_3):#structure to use mennylane with tensorflow
    for i in range(10):
        qml.RY(np.arctan(inputs[i]),wires=i) #first 10 inputs apply in ry
        qml.Hadamard(i)
        qml.RX(np.arctan(inputs[10+i]),wires=i) #the 10-20 inputs apply in rx
    
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[2, 3])
    qml.CNOT(wires=[4, 5])
    qml.CNOT(wires=[6, 7])
    qml.CNOT(wires=[8, 9])
    for i in range(10): 
        qml.RZ(np.arctan(inputs[20+i]),wires=i) #the 20-30 inputs apply in rz
    
    
    #ansatz
    for j,w in enumerate(weights_0): #weights
        qml.RY(w, wires=j)   #apply a layer of ry
        

    # apply different cnots between the ansatz
    qml.CNOT(wires=[0, 1])
    
    qml.CNOT(wires=[4, 3])
    
    qml.CNOT(wires=[3, 2])    
    
    qml.CNOT(wires=[2, 1])
    
    
    #apply RY and Hadamard with the target output of each cx
    list_index = [1,3,5,7,9]
    for j,w in enumerate(weights_1):
        qml.RY(w, wires=list_index[j])      
        

    qml.Hadamard(1)
    qml.Hadamard(3)
    qml.Hadamard(5)
    qml.Hadamard(7)
    qml.Hadamard(9)
    
    # repeat the same ansatz using RX in stead of RY
    for j,w in enumerate(weights_2):
        qml.RX(w, wires=j)
    
    qml.CNOT(wires=[0, 1])
    
    qml.CNOT(wires=[4, 3])
    
    qml.CNOT(wires=[3, 2])    
    
    qml.CNOT(wires=[2, 1])
    
    list_index = [1,3,5,7,9]
    for j,w in enumerate(weights_3):
        qml.RX(w, wires=list_index[j])     
        
    qml.Hadamard(1)
    qml.Hadamard(3)
    qml.Hadamard(5)
    qml.Hadamard(7)
    qml.Hadamard(9)
    return qml.expval(qml.PauliZ(0)), qml.expval(qml.PauliZ(1)), qml.expval(qml.PauliZ(2)), qml.expval(qml.PauliZ(3)), qml.expval(qml.PauliZ(4)),qml.expval(qml.PauliZ(5)), qml.expval(qml.PauliZ(6)), qml.expval(qml.PauliZ(7)), qml.expval(qml.PauliZ(8)), qml.expval(qml.PauliZ(9))
        
weight_shapes = {"weights_0": 10, "weights_1": 5, "weights_2": 10, "weights_3": 5}#,"weights_5": 1,"weights_6": 1,"weights_7": 1,"weights_8": 1,"weights_9": 1}
qlayer = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=10)

In [ ]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Input(shape=()))

model.add(qlayer)
model.add(tf.keras.layers.Reshape((1,10)))
model.add(tf.keras.layers.LSTM(units=100, return_sequences=True))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.LSTM(units=100, return_sequences=True))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.LSTM(units=5))
model.add(tf.keras.layers.Dense(units=1))

In [ ]:
model.compile(
    loss='mean_squared_error',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=[tf.keras.metrics.RootMeanSquaredError()])

model.build()

print(model.summary())

In [ ]:
qnn_history = model.fit(
      tf.constant(x_train), y_train,
      batch_size=30,
      epochs=60)

qnn_results = model.evaluate(tf.constant(x_test), y_test)

Epoch 1/60
 3/27 [==>...........................] - ETA: 36s - loss: 0.3779 - root_mean_squared_error: 0.6147